In [39]:
#encoding algorithm from paper

#create a dictionary with each key being the previous
#nucleotide and the values being the next nucleotide

#Step 1: ASCII --> Trits

#getBinary: string --> list of bitStrings
#from paper: each 12 bit sequence has a 4 bit address and an 8 bit sequence
def getBinary(string):
    bits = []
    for i in range(len(string)):
        #get the bits of each character
        bitsSeq = format(ord(string[i]), 'b')
        #getting the four bit address
        address = '{0:04b}'.format(i)
        #8 bit ascii data
        bits_padding = '{0:08b}'.format(int(bitsSeq,2))
        #appending address/data to bitList
        bits.append(address + bits_padding)
    return bits

def getBinaryfromFile(file):
    with open(file, 'r') as file:
        data = file.read().replace('\n', '')
    return getBinary(data)

#getBinaryfromFile('indpendence.txt')

#bitstoTrits: bitString --> (8-trit) tritString (left-padded with zeros)
def bitstoTrits(bitString):
    trinary = ""
    #first convert bitString to base 10 integer
    baseTen = int(bitString, 2)
    #simple way to convert base 10 integer to base 3 integer
    while (baseTen > 0):
        trinary = str((baseTen)%3) + trinary
        baseTen = baseTen//3
    #return 8-trit trit string left-padded with zeroes
    return '{0:08}'.format(int(trinary))

#getTrits: bitList --> tritList
#usage: getTrits(getBinary(string)) will give you a list w each element
#representing an ASCII char in trits
def getTrits(binList):
    return (list(map(bitstoTrits, binList)))

#getTrits(getBinary('hello world'))
# ['00010212',
#  '00111020',
#  '00211222',
#  '01012110',
#  '01120001',
#  '01210121',
#  '02021022',
#  '02121111',
#  '02222002',
#  '10022100',
#  '10122112']


In [40]:
#Step 2: Get DNA Sequence from trits

#modulation from paper

#getNextNuc: helper function, gets the next nucleotide in the sequence
def getNextNuc(trit, seq):
    #create a dictionary with each key being the previous
    #nucleotide and the values being the next nucleotide
    seqDict = {
    "G": {0: "A", 1: "T", 2: "C"},
    "A": {0: "G", 1: "C", 2: "T"},
    "C": {0: "T", 1: "G", 2: "A"},
    "T": {0: "C", 1: "A", 2: "G"}
    }
    return seqDict[seq[-1]][trit]

#takes in a sequence of trits as a string and returns the correct DNA sequence
def trits2DNA(trits):
    seq = []
    for trit in trits:
        #start with G
        seq.append(getNextNuc(int(trit), "G")) if (not seq) else seq.append(getNextNuc(int(trit), seq[-1]))
    return "".join(seq)

#combine all functions into one master function
#str2DNA: string --> (string representation) DNA Sequence
def str2DNA(binFun, input): 
    seqList = (list(map(trits2DNA, getTrits(binFun(input)))))
    return seqList

#dnaHelloWorld = str2DNA('hello world')
#print("\n".join(dnaHelloWorld))
# AGACTGTG
# AGTACTGA
# AGCGTGCA
# ACTATACT
# ACGCTCTA
# ACACTATA
# ATCACTGC
# ATATACGT
# ATGCAGAT
# TCTGCGAG
# TCGCACGC


In [41]:
#Step 3: Get extension lengths 

#Q: Why is there a discrepancy between the distributions in Fig 2c and Fig 2b? 
def getExtensionLengths(seq):
    #similar dict (key = prev nucleotide, value = dict of all possible nucleotides with *median* extension length)
    #from Fig 2c
    extLenDict = {
    "G": {"A": 4, "C": 2 , "T": 2, },
    "A": {"G": 5 , "C": 3, "T": 3 },
    "C": {"T": 4, "G": 6 , "A": 11},
    "T": {"C": 2, "A": 3 , "G": 4}
    }
    extSeq = ""
    count = 0
    for s in seq:
        if (not extSeq):
            extSeq += s * extLenDict["G"][str(s)]
            count += extLenDict["G"][str(s)]
        else:
             nextExt = extLenDict[extSeq[-1]][s]
             extSeq += s * nextExt
             count += nextExt
    return (extSeq, count)

#getInfoIndex: gets the bits/nt rate for an ASCII string
#extLenFun = extenstion length function used 
#modFun = function used for ascii --> DNA sequence
#ASCII string --> float rate
def getInfoIndex(extLenFun, modFun, binFun, string):
    extendedDNA = list((map(extLenFun, modFun(binFun, string))))
    #get total # nucleotides
    nucTotal = sum(length for seq,length in extendedDNA)
    #get GC content 
    gcCount = 0
    for dnaSeq in extendedDNA: 
        for nuc in dnaSeq[0]:
            if nuc is "G" or nuc is "C":
                gcCount += 1
    #get total # of bits
    bitsTotal = sum(len(bits) for bits in binFun(string))
    return (str(bitsTotal/nucTotal)+"bits/nt", "total bits: " + str(bitsTotal), "total nucleotides: " + str(nucTotal), "GC content: " + str(gcCount/nucTotal))



In [42]:
#Trying a different modulation - basically each transition between base pairs counts for 2 trits
# newModDict = {
#     "00": "AT",
#     "01": "AG",
#     "02": "AC",
#     "10": "TA",
#     "11": "TG",
#     "12": "TC",
#     "20": "GA",
#     "21": "GT",
#     "22": "GC"
#     }
#i.e. if you wanted to encode 002012, it would be ATGATC
#Pros: You are minimizing the number of C transitions
#Cons: As you can probably tell, there's an issue because you'd get repeat bases, and it would be hard to distinguish if it's 
#one base being extended or two (i.e. encoding 0011 would be ATTG)

def newModulation(trits):
    newModDict = {
    "00": "AT",
    "01": "AG",
    "02": "AC",
    "10": "TA",
    "11": "TG",
    "12": "TC",
    "20": "GA",
    "21": "GT",
    "22": "GC"
    }
    seq = ""

    for i in range(0, len(trits)-1, 2):
        seq += newModDict[(trits[i]+trits[i+1])]
    return seq

def str2DNA2(binFun, input): 
    seqList = (list(map(newModulation, getTrits(binFun(input)))))
    return seqList

def getExtensionLengths2(seq):
    #similar dict (key = prev nucleotide, value = dict of all possible nucleotides with *median* extension length)
    #from Fig 2c
    #I know it wouldn't work, but I included same-base transitions and made them 3 (I guess it could work if you could TELL that it was an extension of two bases)
    extLenDict = {
    "G": {"A": 4, "C": 2 , "T": 2, "G": 3},
    "A": {"G": 5 , "C": 3, "T": 3, "A": 3},
    "C": {"T": 4, "G": 6 , "A": 11, "C": 3},
    "T": {"C": 2, "A": 3 , "G": 4, "T": 3}
    }
    extSeq = ""
    count = 0
    for s in seq:
        if (not extSeq):
            extSeq += s
            count += 1
        else:
             nextExt = extLenDict[extSeq[-1]][s]
             extSeq += s * nextExt
             count += nextExt
    return (extSeq, count)



In [43]:
#Examples: 
#Example 1: "hello world"

#orig mapping: 
print("Orig. DNA Sequence:\n" + str("\n".join(str2DNA(getBinary, "hello world"))))

print(str(getInfoIndex(getExtensionLengths, str2DNA, getBinary, "hello world")))


Orig. DNA Sequence:
AGACTGTG
AGTACTGA
AGCGTGCA
ACTATACT
ACGCTCTA
ACACTATA
ATCACTGC
ATATACGT
ATGCAGAT
TCTGCGAG
TCGCACGC
('0.3848396501457726bits/nt', 'total bits: 132', 'total nucleotides: 343', 'GC content: 0.41690962099125367')


In [46]:
#new mapping: 
print("New DNA Sequence:\n" + str("\n".join(str2DNA2(getBinary, "hello world"))))

print(str(getInfoIndex(getExtensionLengths2, str2DNA2, getBinary, "hello world")))

New DNA Sequence:
ATAGACTC
ATTGTAGA
ATGTTCGC
AGAGGTTA
AGTCATAG
AGGTAGGT
ACACTAGC
ACTCTGTG
ACGCGAAC
TAACGTAT
TATCGTTC
('0.4631578947368421bits/nt', 'total bits: 132', 'total nucleotides: 285', 'GC content: 0.48771929824561405')


In [47]:
#Example 2 with more data (sequences are not a standard value right now)
#getBinaryfromFile('indpendence.txt')
#if you wanna see the DNA sequences: 
#print("Orig DNA Sequence:\n" + str("\n".join(str2DNA(getBinaryfromFile, 'indpendence.txt'))))
#print("New DNA Sequence:\n" + str("\n".join(str2DNA2(getBinaryfromFile, 'indpendence.txt'))))

print("Orig mapping: "+ str(getInfoIndex(getExtensionLengths, str2DNA, getBinaryfromFile, 'indpendence.txt')))
print("New mapping: " + str(getInfoIndex(getExtensionLengths2, str2DNA2, getBinaryfromFile, 'indpendence.txt')))


Orig mapping: ('0.3933181920970612bits/nt', 'total bits: 160016', 'total nucleotides: 406836', 'GC content: 0.4475587214504125')
New mapping: ('0.4652534069135558bits/nt', 'total bits: 160016', 'total nucleotides: 343933', 'GC content: 0.43012156437445664')
